# Question 1: Web Scraping

In [1]:
#Importing required libraries
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
#Getting web link and printing the details

idmp_link="https://www.imdb.com/chart/top"  #idmp link
r=requests.get(idmp_link)                   
page=r.text
soup=BeautifulSoup(page,"lxml")

#to check the link content
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
  </script>
  <link href="https://www.imdb.com/chart/top" rel="canonical"/>
  <meta content="http://www.imdb.com/chart/

In [3]:
#Extracting required columns
movie_details=soup.select("td.titleColumn")      #fetch movie details       
movie_stars = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]  #get acotrs and directors
imdb_rating = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')] #to get ratings

#empty list
imdb = []

In [4]:
#Combining all movie details in a list
for index in range(0, len(movie_details)):             #run loop for all movie details
    data = movie_details[index].get_text()
    imdb_movie = (' '.join(data.split()).replace('.', ''))
    title = imdb_movie[len(str(index))+1:-7]               #to get the title of movies
    release_year = re.search('\((.*?)\)', data).group(1)   #to get rlease years
    data_final = {"title": title,
            "release_year": release_year,
            "movie_stars": movie_stars[index],
            "imdb_rating": imdb_rating[index]}
    imdb.append(data_final)
imdb

[{'title': 'The Shawshank Redemption',
  'release_year': '1994',
  'movie_stars': 'Frank Darabont (dir.), Tim Robbins, Morgan Freeman',
  'imdb_rating': '9.220677150970296'},
 {'title': 'The Godfather',
  'release_year': '1972',
  'movie_stars': 'Francis Ford Coppola (dir.), Marlon Brando, Al Pacino',
  'imdb_rating': '9.147355656125736'},
 {'title': 'The Godfather: Part II',
  'release_year': '1974',
  'movie_stars': 'Francis Ford Coppola (dir.), Al Pacino, Robert De Niro',
  'imdb_rating': '8.98086931133095'},
 {'title': 'The Dark Knight',
  'release_year': '2008',
  'movie_stars': 'Christopher Nolan (dir.), Christian Bale, Heath Ledger',
  'imdb_rating': '8.974168828510042'},
 {'title': '12 Angry Men',
  'release_year': '1957',
  'movie_stars': 'Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb',
  'imdb_rating': '8.94047323508882'},
 {'title': "Schindler's List",
  'release_year': '1993',
  'movie_stars': 'Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes',
  'imdb_rating': '8.911491

In [5]:
#creating dataframe
df=pd.DataFrame(imdb)

#To divide movie_star column into 2 columns- director names and actor names
df['director']=df.movie_stars.str.split(",").str[0]
df['director']=df.director.replace({"\(dir.\)":''},regex=True)
df['actors']=df.movie_stars.str.split(",").str[1:3]

#round ratings upto 1 decimal
df['imdb_rating'] = df['imdb_rating'].apply(lambda x: round(float(x), 1))

#rearrange columns in required fromat
df=df.reindex(columns=['title','director','actors','release_year','imdb_rating'])

#show top 10 movies
df.head(10)

,title,director,actors,release_year,imdb_rating
0,The Shawshank Redemption,Frank Darabont,"[ Tim Robbins, Morgan Freeman]",1994,9.2
1,The Godfather,Francis Ford Coppola,"[ Marlon Brando, Al Pacino]",1972,9.1
2,The Godfather: Part II,Francis Ford Coppola,"[ Al Pacino, Robert De Niro]",1974,9.0
3,The Dark Knight,Christopher Nolan,"[ Christian Bale, Heath Ledger]",2008,9.0
4,12 Angry Men,Sidney Lumet,"[ Henry Fonda, Lee J. Cobb]",1957,8.9
5,Schindler's List,Steven Spielberg,"[ Liam Neeson, Ralph Fiennes]",1993,8.9
6,The Lord of the Rings: The Return of the King,Peter Jackson,"[ Elijah Wood, Viggo Mortensen]",2003,8.9
7,Pulp Fiction,Quentin Tarantino,"[ John Travolta, Uma Thurman]",1994,8.8
8,"The Good, the Bad and the Ugly",Sergio Leone,"[ Clint Eastwood, Eli Wallach]",1966,8.8
9,The Lord of the Rings: The Fellowship of the ...,Peter Jackson,"[ Elijah Wood, Ian McKellen]",2001,8.8


# Question 2: XML and XPath

# 2. 1

In [6]:
#importing required libraries
from xml.dom import minidom
import pandas as pd

In [7]:
#to get the xml file
file=minidom.parse("C:/Users/apoor/Desktop/Apoorva/Sem 1/IDMP/Assignments/PS3/recipes.xml")

In [8]:
#to fetch the required details

df = pd.DataFrame()                    #creating dataframe
recipes=file.getElementsByTagName('recipe')  #get all recipe elements

#creating empty list
titles=[]
ingredients=[]
calorie=[]

#getting details in list
for recipe in recipes:
    title=recipe.getElementsByTagName('title')[0].firstChild.data
    titles.append(recipe.getElementsByTagName('title')[0].firstChild.data)

    ingredient_name=[]
    ingredient=recipe.getElementsByTagName('ingredient')
    for name in ingredient:
        ingredient_name.append(name.getAttribute('name'))
    ingredients.append(ingredient_name)  
    
    nutrition=recipe.getElementsByTagName('nutrition')
    for name in nutrition:
        calories=name.getAttribute('calories')
    calorie.append(calories)
        
     #print recipe titles, ingredients and calories   
    print('title: ',title,', ingredient name:',ingredient_name,', calories:',calories)

#getting details from list to dataframe    
df['titles']=titles
df['ingredients']=ingredients
df['calories']=calorie

#printing above details in dataframe
df

title:  Beef Parmesan with Garlic Angel Hair Pasta , ingredient name: ['beef cube steak', 'onion, sliced into thin rings', 'green bell pepper, sliced in rings', 'Italian seasoned bread crumbs', 'grated Parmesan cheese', 'olive oil', 'spaghetti sauce', 'shredded mozzarella cheese', 'angel hair pasta', 'minced garlic', 'butter'] , calories: 1167
title:  Ricotta Pie , ingredient name: ['filling', 'ricotta cheese', 'eggs', 'white sugar', 'vanilla extract', 'semisweet chocolate chips', 'dough', 'flour', 'baking powder', 'white sugar', 'shortening', 'eggs, lightly beaten', 'vanilla extract', 'milk'] , calories: 349
title:  Linguine Pescadoro , ingredient name: ['linguini pasta', 'sauce', 'olive oil', 'minced cloves of garlic', 'Italian seasoning', 'dried thyme', 'crushed red pepper flakes', 'crushed tomatoes', 'black olives, drained', 'whole baby clams', 'minced clams, with juice', 'small salad shrimp', 'scallops', 'lemon zest', 'salt', 'ground black pepper'] , calories: 532
title:  Zuppa In

,titles,ingredients,calories
0,Beef Parmesan with Garlic Angel Hair Pasta,"[beef cube steak, onion, sliced into thin ring...",1167
1,Ricotta Pie,"[filling, ricotta cheese, eggs, white sugar, v...",349
2,Linguine Pescadoro,"[linguini pasta, sauce, olive oil, minced clov...",532
3,Zuppa Inglese,"[egg yolks, milk, Savoiardi biscuits, sugar, A...",612
4,Cailles en Sarcophages,"[pastry, chilled unsalted butter, flour, salt,...",8892


# 2.2

In [9]:
#importing required libraries
from lxml import etree

#to get the xml file
tree=etree.parse("C:/Users/apoor/Desktop/Apoorva/Sem 1/IDMP/Assignments/PS3/recipes.xml")

In [10]:
#(a) find the titles of all recipes
results=tree.xpath('//recipe/title')
for result in results:
    print(result.text)

Beef Parmesan with Garlic Angel Hair Pasta
Ricotta Pie
Linguine Pescadoro
Zuppa Inglese
Cailles en Sarcophages


In [11]:
#(b) Find the titles of recipes that use olive oil.
results=tree.xpath('//ingredient[@name = "olive oil"]/../title/text()') + tree.xpath('//ingredient[@name = "olive oil"]/../../title/text()')
#since one recipe has olive oil in ingredient and other recipe has olive oil as composite ingredient 
for result in results:
   print(result)

Beef Parmesan with Garlic Angel Hair Pasta
Linguine Pescadoro


In [12]:
#(c) Find the titles of all recipes with less than 500 calories.
results=tree.xpath("//recipe[nutrition/@calories<500]/title")
for result in results:
    print(result.text)

Ricotta Pie


In [13]:
#(d) Find the amount of sugar needed for Zuppa Inglese.
results=tree.xpath("//title[contains(.,'Zuppa Inglese')]/../ingredient[@name='sugar']/@amount")
for result in results:
    print(result)

0.75


In [14]:
#(e) Find the titles of all recipes that require 4 steps.
results=tree.xpath("//recipe/preparation[count(./step)=4]/../title")
for result in results:
    print(result.text)

Beef Parmesan with Garlic Angel Hair Pasta
Ricotta Pie


In [15]:
#(f) Find the names of all ingredients that are used to make other ingredients
results=tree.xpath("//recipe//ingredient/ingredient/@name" )
for result in results:
    print(result)

ricotta cheese
eggs
white sugar
vanilla extract
semisweet chocolate chips
flour
baking powder
white sugar
shortening
eggs, lightly beaten
vanilla extract
olive oil
minced cloves of garlic
Italian seasoning
dried thyme
crushed red pepper flakes
crushed tomatoes
black olives, drained
whole baby clams
minced clams, with juice
small salad shrimp
scallops
lemon zest
salt
ground black pepper
chilled unsalted butter
flour
salt
ice water
baked chicken
marinated chicken
small chickens, cut up
Herbes de Provence
dry white wine
orange juice
minced garlic
truffle oil
stock
chicken wings, giblets, and kidney
onions, peeled
carrots, peeled and cut lengthwise
celery, cut lengthwise
bay leaf
small bunch parsley
whole peppercorns
salt
sauteed mushrooms
white button mushrooms
butter
dry white wine
minced garlic
minced shallots
sauce
chicken juices
mushroom juices
sherry
flour
butter


In [16]:
#(g) Find the names of all ingredients for which you need other ingredients
results=tree.xpath("//recipe//ingredient[ingredient]/@name" )
for result in results:
    print(result)

filling
dough
sauce
pastry
filling
baked chicken
marinated chicken
stock
sauteed mushrooms
sauce


In [17]:
#(h) Find the names of the first three ingredients in each recipe
results = tree.xpath("//recipe")
for result in results :
  for j in (result.xpath("ingredient")[:3]):
    print(j.get('name'))

beef cube steak
onion, sliced into thin rings
green bell pepper, sliced in rings
filling
dough
milk
linguini pasta
sauce
egg yolks
milk
Savoiardi biscuits
pastry
filling
package phyllo dough


# Question 3: Web API

In [18]:
#Importing required libraries
import requests
from requests import get
import json
import pandas as pd

In [19]:
#creating class to get the valid link for routes
class route:
    def __init__(self,key):
        self.address_1 = None                 #from location
        self.address_2 = None                 #to location
        self.key = key
        self.url = r"http://open.mapquestapi.com/directions/v2/route?key=KEY".replace('KEY',key) #url link
        self.query = None

        #method to get the valid address by adding and removing non alphabets in address
    def addressString(self,first_Line, second_line = None, city = None, state = None, pincode = None):
        address = [first_Line, second_line, city, state, pincode]
        address = [i.strip() for i in address if (i!=None) ]
        address = [i.strip() for i in address if i.strip()!='']
        address = ', '.join(address)
        return address.replace(" ","+")

    #method to get complete from address
    def add_address_1(self,first_Line, second_line = None, city = None, state = None, pincode = None):
        self.address_1 = self.addressString(first_Line, second_line, city, state, pincode)
        
    #method to get complete to address
    def add_address_2(self,first_Line, second_line = None, city = None, state = None, pincode = None):
        self.address_2 = self.addressString(first_Line, second_line, city, state, pincode)
    
    #method to get the complete url
    def final_URL(self):
        if (self.address_1!=None) & (self.address_2 != None):
            from_address = ''.join(["from=",self.address_1])
            to_address = ''.join(["to=",self.address_2])
            self.query = '&'.join([self.url, from_address,to_address])
            return self.query
        else:
            return None
    
    #to call the required route link
    def directions(self):
        return requests.get(self.final_URL())

In [20]:
#defining KEY and locations
API_KEY = 'fuHoJbihAXqtrbxL38vN2YKcgZ3hBCNM'          #created KEY
result = route(API_KEY)                               #creating class object 

#taking from and to locations from the user:
from_loc=input("Enter from location: ")
to_loc=input("Enter to location: ")
result.add_address_1(from_loc)      #from location= Northeastern University
result.add_address_2(to_loc)     #to location= Faneuil Hall Marketplace

result.final_URL()
json_out= result.directions()

Enter from location: Northeastern University
Enter to location: Faneuil Hall Marketplace


In [21]:
#creating dataframe 
df = pd.DataFrame(json_out.json()['route']['legs'][0]['maneuvers'])

In [22]:
#getting required columns
final=df.loc[:,['narrative','distance','time']]

#renaming columns
final=final.rename(columns={"narrative":"instruction","distance":"distance (mi)","time":"time (s)"})

#printing final dataframe
final

,instruction,distance (mi),time (s)
0,Start out going northeast toward Gainsborough St.,0.199,52
1,Turn left onto Gainsborough St.,0.105,35
2,Turn right onto SR 9 E/Huntington Ave. Continu...,0.826,81
3,SR 9 E becomes Stuart St.,0.605,95
4,Turn left onto Washington St.,0.108,25
5,Turn right onto Essex St.,0.235,47
6,Merge onto I-93/US-1/SR 3/John FFitzgerald Expy.,0.773,79
7,Take EXIT 23 toward Government Center.,0.181,30
8,Turn left onto John FFitzgerald Surface Rd.,0.091,13
9,Turn right onto Clinton St.,0.051,6
